In [ ]:
import os
import re
import json
import asyncio
from pathlib import Path

from dotenv import load_dotenv
from openai import AsyncOpenAI

# ─────────────────────────────────────────────
# 1️⃣ Load API key from .env and init client
# ─────────────────────────────────────────────
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY is missing in .env")

client = AsyncOpenAI(api_key=api_key)

# ─────────────────────────────────────────────
# 2️⃣ Resolve AUDIO_DIR robustly
#    Looks for the folder "Audio Speech Translator AI Agent"
#    and uses its "audio" subfolder.
# ─────────────────────────────────────────────
AUDIO_FOLDER_NAME = "Audio Speech Translator AI Agent"
AUDIO_SUBFOLDER = "audio"

cwd = Path.cwd()
project_root = None

# Try current dir and all parents
for p in [cwd, *cwd.parents]:
    if (p / AUDIO_FOLDER_NAME).exists():
        project_root = p
        break

if project_root is None:
    # Fallback: assume we're already inside the agent folder
    project_root = cwd

AUDIO_DIR = project_root / AUDIO_FOLDER_NAME / AUDIO_SUBFOLDER
AUDIO_DIR = AUDIO_DIR.resolve()

print(f"🎯 Audio folder resolved to: {AUDIO_DIR}")
print("📄 Files found:", list(AUDIO_DIR.glob("*.*")))

# ─────────────────────────────────────────────
# 3️⃣ Helper: resolve audio path
# ─────────────────────────────────────────────
def get_audio_path(file_name: str) -> Path:
    name = file_name.strip()
    if not name:
        raise ValueError("❌ Empty audio filename provided")

    audio_path = (AUDIO_DIR / name).resolve()

    if not audio_path.exists():
        raise FileNotFoundError(f"❌ Audio file not found: {audio_path}")

    return audio_path

# ─────────────────────────────────────────────
# 4️⃣ Transcription with gpt-4o-transcribe
# ─────────────────────────────────────────────
async def transcribe_audio(file_path: str) -> str:
    audio_path = Path(file_path).resolve()
    with open(audio_path, "rb") as audio:
        transcription = await client.audio.transcriptions.create(
            model="gpt-4o-transcribe",
            file=audio,
        )
    return transcription.text

# ─────────────────────────────────────────────
# 5️⃣ Language + emotion detection
# ─────────────────────────────────────────────
async def detect_language_and_emotion(text: str) -> dict:
    """
    Returns a JSON dict: { "language": "...", "emotion": "..." }
    """
    prompt = (
        "You are an assistant that detects language and emotional tone of a short text.\n"
        "Return a JSON object with exactly two keys:\n"
        "  - language: the language name in English (e.g., 'French', 'Arabic')\n"
        "  - emotion: one or two words describing the dominant emotion (e.g., 'happy', 'sad', 'angry').\n\n"
        f"Text:\n```{text}```"
    )

    resp = await client.chat.completions.create(
        model="gpt-4.1-mini",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "You output only valid JSON."},
            {"role": "user", "content": prompt},
        ],
    )

    content = resp.choices[0].message.content
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        # Fallback: try to extract something even if not valid JSON
        data = {"language": "unknown", "emotion": "unknown"}

    return {
        "language": data.get("language", "unknown"),
        "emotion": data.get("emotion", "unknown"),
    }

# ─────────────────────────────────────────────
# 6️⃣ Translation
# ─────────────────────────────────────────────
async def translate_text(text: str, target_language: str = "English") -> str:
    """
    Translate `text` into `target_language`.
    """
    prompt = (
        f"Translate the following text into {target_language}. "
        "Only return the translated text, no explanations.\n\n"
        f"Text:\n```{text}```"
    )

    resp = await client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": prompt},
        ],
    )

    return resp.choices[0].message.content.strip()

# ─────────────────────────────────────────────
# 7️⃣ Orchestrator: process_audio_translation
# ─────────────────────────────────────────────
async def process_audio_translation(file_name: str, target_language: str = "English"):
    # 0) Resolve audio path
    file_path = get_audio_path(file_name)
    print(f"🎧 Using audio file: {file_path}")

    # 1) Transcribe
    print("🎧 Transcribing…")
    transcript = await transcribe_audio(str(file_path))
    print(f"📝 Transcript:\n{transcript}\n")

    # 2) Language & emotion
    print("🧠 Detecting language & emotion…")
    analysis = await detect_language_and_emotion(transcript)
    language = analysis["language"]
    emotion = analysis["emotion"]
    print(f"🌍 Detected language: {language}")
    print(f"💓 Detected emotion: {emotion}\n")

    # 3) Translate
    print(f"🌐 Translating into {target_language}…")
    translated = await translate_text(transcript, target_language=target_language)
    print(f"💬 Translation ({target_language}):\n{translated}\n")

    # 4) Return everything as a dict (useful if you want to consume programmatically)
    return {
        "file": str(file_path),
        "transcript": transcript,
        "language": language,
        "emotion": emotion,
        "translation_language": target_language,
        "translation": translated,
    }


🎯 Audio folder resolved to: /Users/JABERI/Documents/Dev/AI Agent/KodeKloud/AI Agent/ResumeScan-SpeechTranslate-AI-Agents/Audio Speech Translator AI Agent/audio
📄 Files found: [PosixPath('/Users/JABERI/Documents/Dev/AI Agent/KodeKloud/AI Agent/ResumeScan-SpeechTranslate-AI-Agents/Audio Speech Translator AI Agent/audio/arabic_sample.mp3'), PosixPath('/Users/JABERI/Documents/Dev/AI Agent/KodeKloud/AI Agent/ResumeScan-SpeechTranslate-AI-Agents/Audio Speech Translator AI Agent/audio/french_sample.mp3'), PosixPath('/Users/JABERI/Documents/Dev/AI Agent/KodeKloud/AI Agent/ResumeScan-SpeechTranslate-AI-Agents/Audio Speech Translator AI Agent/audio/english_sample.mp3')]


In [ ]:
# ─────────────────────────────────────────────
# 8️⃣ Example calls (Notebook usage)
# ─────────────────────────────────────────────
# Make sure these files exist in:
#   Audio Speech Translator AI Agent/audio/
#
# await process_audio_translation("english_sample.mp3")
# await process_audio_translation("french_sample.mp3")
# await process_audio_translation("arabic_sample.mp3")

await process_audio_translation("french_sample.mp3")


🎧 Using audio file: /Users/JABERI/Documents/Dev/AI Agent/KodeKloud/AI Agent/ResumeScan-SpeechTranslate-AI-Agents/Audio Speech Translator AI Agent/audio/french_sample.mp3
🎧 Transcribing…


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************upIA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}